Flow:
Interleaving salting ->  Preorder and Inorder concatenation of Binary tree -> Artificial Neural Network 

# Installing dependencies

In [1]:
import numpy as np
import copy
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential 
from tensorflow.keras import layers
from tensorflow.keras import initializers
from itertools import chain
import collections
from collections import deque

# Input data

In [2]:
# input_data = "HDNKd gKLC LFKfg  fgGAUFfdsa fCMDG"
# salt = "NBPFQLLpkqIQHIz10TGG"
input_data = "abcdefghijklmnopqrstuvwxyz"
salt = "123456789"

# Interleaving Function

## Encryption

In [3]:
def interleave(data1, data2):
    
    original_data2 = copy.deepcopy(data2)
    
    while(len(data2) < len(data1)):
        data2 += original_data2
    
    lvl1 = [''] * len(data1) * 2 
    lvl1[::2] = data1[:len(data1)]
    lvl1[1::2] = data2[:len(data1)]
    
    return lvl1

In [4]:
lvl1 = interleave(input_data, salt)

In [5]:
str(lvl1)

"['a', '1', 'b', '2', 'c', '3', 'd', '4', 'e', '5', 'f', '6', 'g', '7', 'h', '8', 'i', '9', 'j', '1', 'k', '2', 'l', '3', 'm', '4', 'n', '5', 'o', '6', 'p', '7', 'q', '8', 'r', '9', 's', '1', 't', '2', 'u', '3', 'v', '4', 'w', '5', 'x', '6', 'y', '7', 'z', '8']"

## Decryption

In [6]:
def decrypt_interleave(interleaved_data):
    
    data1 = interleaved_data[::2]
    data2 = interleaved_data[1::2]
    
    return data1, data2

In [7]:
data1, data2 = decrypt_interleave(interleaved_data = lvl1)

In [8]:
print(data1, data2)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'] ['1', '2', '3', '4', '5', '6', '7', '8', '9', '1', '2', '3', '4', '5', '6', '7', '8', '9', '1', '2', '3', '4', '5', '6', '7', '8']


# Binary Tree code

## functions to create and traverse trees

In [9]:
class Node:
    def __init__(self, key, left = None, right = None):
        self.data = key
        self.left = left
        self.right = right

In [10]:
# Creates a binary tree using level order traversal, 2 nodes per branch
def makeTree(s): 
    
    root=Node(s[0])
    queue = []
 
    queue.append((root,0))
 
    while(len(queue) > 0):
        sz=len(queue)
        for i in range(0,sz):
            node,index = queue.pop(0)
    
            if (index*2+1) < len(s):
                node.left=Node(s[index*2+1])
                queue.append((node.left,index*2+1))
    
            if (index*2+2) < len(s):
                node.right=Node(s[index*2+2])
                queue.append((node.right,index*2+2))
    
    return root

In [11]:
# Returns the level order traversal of the tree
def getLevelOrder(root):
    
    if root is None:
        return

    queue = []
 
    queue.append(root)

    levelorder = []
    
    while(len(queue) > 0):
        
        sz=len(queue)
        
        for i in range(0,sz):
            
            levelorder.append(queue[0].data)
            node = queue.pop(0)
    
            if node.left is not None:
                queue.append(node.left)

            if node.right is not None:
                queue.append(node.right)
        
    return levelorder

In [12]:
# Returns the preorder traversal of the tree
def getPreorder(root):
    
    preorder.append(root.data)
    
    if root.left is not None:
        getPreorder(root.left)
    
    if root.right is not None:
        getPreorder(root.right)

In [13]:
# Returns the inorder traversal of the tree
def getInorder(root):
    
    if root.left is not None:
        getInorder(root.left)
    
    inorder.append(root.data)
    
    if root.right is not None:
        getInorder(root.right)

In [14]:
# Recursive function to construct a binary tree from a given inorder and preorder sequence
def construct(start, end, preorder, pIndex, d):
 
    if start > end:
        return None, pIndex
 
    root = Node(preorder[pIndex])
    pIndex = pIndex + 1
 
    index = d[root.data]
 
    root.left, pIndex = construct(start, index - 1, preorder, pIndex, d)
 
    root.right, pIndex = construct(index + 1, end, preorder, pIndex, d)
 
    return root, pIndex

In [15]:
# Construct a binary tree from inorder and preorder traversals.
def constructTree(inorder, preorder):
 
    d = {}
    for i, e in enumerate(inorder):
        d[e] = i

    pIndex = 0
 
    return construct(0, len(inorder) - 1, preorder, pIndex, d)[0]

In [16]:
# modifies duplicates so that a binary tree could be constructed using preorder and inorder traversal. 
# Eg. ['a', 'a', 'b', a', 'b'] would become ['a', 'a*', 'b', a**', 'b*']

def modify_duplicates(input_list):
    
    for index, i in enumerate(input_list):
        
        if i in input_list[index+1:]:
            num = 1
        
            for j in range(index+1, len(input_list)):
                
                if(input_list[j] == i):
                    append_str = '*'*num
                    input_list[j] = f'{i}{append_str}'
                    num += 1
                    
    return input_list

In [17]:
# This function removes "*" from the output list as "*" were added to make sure that each element is a unique element 

def remove_stars(input_list):
    
    for i in range(0, len(input_list)):
        if '*' in input_list[i]:
            input_list[i] = input_list[i].split('*')[0]
            
    return input_list

## Binary tree encryption function

In [24]:
def encrypt_binary_tree(lvl1):
    
    s = modify_duplicates(input_list = lvl1)
    
    root = makeTree(lvl1)
    
#     levelorder = getLevelOrder(root)
    
    getInorder(root)
    
    getPreorder(root)
        
#     lvl2 = interleave(inorder, preorder)
    
#     lvl2 = remove_stars(lvl2)
    
    return inorder, preorder

In [25]:
inorder = []
preorder = []
inorder, preorder = encrypt_binary_tree(lvl1)

## Binary tree decryption function

In [27]:
def decrypt_binary_tree(inorder, preorder):
    
    root2 = constructTree(inorder, preorder)
    
    levelorder = getLevelOrder(root2)
    
    lvl1 = remove_stars(input_list = levelorder)
    
    return lvl1

In [28]:
lvl1 = decrypt_binary_tree(inorder, preorder)

In [29]:
str(lvl1)

"['a', '1', 'b', '2', 'c', '3', 'd', '4', 'e', '5', 'f', '6', 'g', '7', 'h', '8', 'i', '9', 'j', '1', 'k', '2', 'l', '3', 'm', '4', 'n', '5', 'o', '6', 'p', '7', 'q', '8', 'r', '9', 's', '1', 't', '2', 'u', '3', 'v', '4', 'w', '5', 'x', '6', 'y', '7', 'z', '8']"

# Artificial Neural Network

## Functions for encrypting Artificial Neural Networks

In [30]:
inorder_lvl2, preorder_lvl2 = copy.deepcopy(inorder), copy.deepcopy(preorder)

In [31]:
def get_ann_input(data_lvl2):
    
    data_ann = []

    for i in data_lvl2:

        if '*' in i:
            l = i.split('*')
            num = ord(l[0])
            num += 256*(len(l)-1)
            data_ann.append(num)

        else:
            data_ann.append(ord(i))
            
    return data_ann

In [32]:
def slice_input(input_array, n):
    
    sliced_array = []
    
    for i in range(0, int(len(input_array)/n)):
        
        sliced_array.append(input_array[n*i:n*(i+1)])
        
    sliced_array.append(input_array[n*(i+1):])
    
    while(len(sliced_array[-1]) < n):
        sliced_array[-1].append(-1)
        
    return sliced_array

In [33]:
if not os.path.exists(os.path.join(os.getcwd(), 'weights')):
    os.makedirs(os.path.join(os.getcwd(), 'weights'))

In [34]:
def create_model_initial(n, user_id):

    layer = layers.Dense(
        units=64,
        kernel_initializer=initializers.RandomNormal(stddev=0.01),
        bias_initializer=initializers.Zeros()
    )  
    model = Sequential([
        layers.Dense(units = n, input_shape = (n,), kernel_initializer=initializers.RandomNormal(mean=0.4, stddev=1.0, seed=user_id*3)),
        layers.Dense(units = n, kernel_initializer=initializers.RandomNormal(mean=0.4, stddev=1.0, seed=user_id*3+1)), 
        layers.Dense(units = n, kernel_initializer=initializers.RandomNormal(mean=0.4, stddev=1.0, seed=user_id*3+2))
    ])
    
    model_path = os.path.join(os.getcwd(), 'weights', f"standard_encryption_weights_{user_id}.h5")
    model.save(model_path)
    
    return model_path

In [35]:
def get_predictions(sliced_array, n):
    
    lvl3 = []
    sliced_array = np.array(sliced_array)
    
    for array in sliced_array:
        
        predictions = model.predict(np.array(array).reshape(1,n))
        lvl3.append(predictions)
        
    return lvl3

## Encryption using Artificial Neural Network

In [36]:
inorder_ann = get_ann_input(data_lvl2 = inorder_lvl2) # Converts into numeric form

In [37]:
preorder_ann = get_ann_input(data_lvl2 = preorder_lvl2) # Converts into numeric form

In [69]:
inorder_ann_sliced = slice_input(inorder_ann, n = 16) # slices the array into arrays of size n, 
# the last remaining values of the last array are filled with -1

In [71]:
preorder_ann_sliced = slice_input(preorder_ann, n = 16) # slices the array into arrays of size n, 
# the last remaining values of the last array are filled with -1

In [72]:
model_path = create_model_initial(n=16, user_id=1)
model = tf.keras.models.load_model(model_path)

In [73]:
lvl3_inorder = get_predictions(inorder_ann_sliced, n=16)

In [74]:
lvl3_preorder = get_predictions(preorder_ann_sliced, n=16)

## Functions for decrypting Artificial Neural Network

In [76]:
def get_input_ann(model_path, lvl3, n):
    
    model = tf.keras.models.load_model(model_path)
    
    n_layers = len(model.layers)
    
    dependent_var = lvl3
    for i in range(1, n_layers+1):
        
        dependent_var = np.linalg.solve(model.layers[-i].get_weights()[0].T, dependent_var.reshape(n, 1))
        
    return dependent_var

In [82]:
def decrypt_ann(model_path, lvl3, n):
    
    input_ann = []
    
    for prediction in lvl3:
        
        inp = get_input_ann(model_path, prediction, n)
        for i in range(0, len(inp)):
            if(inp[i][0]<0):
                inp[i][0] = -1
            else:
                inp[i][0] = int(inp[i][0]+0.5) 
        input_ann.append(inp)
        
    return input_ann

In [95]:
def get_lvl2(data_array):
    
    lvl2_decrypted = []
    
    for array in data_array:
                
        for i in array:
            
#             print(i)
            i = int(i)
            if(i == -1):
                break
            
            elif(i>256):
                n_star = int(i/256)
                stars = '*' * n_star
                char = chr(i%256)
                lvl2_decrypted.append(f'{char}{stars}')
            
            else:
                lvl2_decrypted.append(chr(i))
            
    lvl2_decrypted = np.array(lvl2_decrypted).reshape(-1)
    
    return lvl2_decrypted

## Decrypting Artificial Neural Network

In [87]:
inorder_ann_input = decrypt_ann(model_path, lvl3 = lvl3_inorder, n = 16)

In [83]:
preorder_ann_input = decrypt_ann(model_path, lvl3 = lvl3_preorder, n = 16)

In [96]:
inorder_lvl2_decrypted = get_lvl2(inorder_ann_input)

In [99]:
preorder_lvl2_decrypted = get_lvl2(preorder_ann_input)

# Combining functions

## Encryption Function

In [114]:
# input_data = "HDNKd gKLC LFKfg  fgGAUFfdsa fCMDG"
# salt = "NBPFQLLpkqIQHIz10TGG"
input_data = "abcdefghijklmnopqrstuvwxyz"
salt = "123456789"

In [59]:
def encrypt(input_data, salt, n, user_id): # n represents the number of nodes in the layer of the ANN
    
    lvl1 = interleave(input_data, salt)
    
    inorder, preorder = encrypt_binary_tree(lvl1)
    inorder_lvl2, preorder_lvl2 = copy.deepcopy(inorder), copy.deepcopy(preorder)
    
    inorder_ann = get_ann_input(data_lvl2 = inorder_lvl2)
    preorder_ann = get_ann_input(data_lvl2 = preorder_lvl2)
    inorder_ann_sliced = slice_input(inorder_ann, n)
    preorder_ann_sliced = slice_input(preorder_ann, n = 16)
    model_path = os.path.join(os.getcwd(), 'weights', f"standard_encryption_weights_{user_id}.h5")
    is_exists = os.path.exists(model_path)
    if not is_exists:
        model_path = create_model_initial(n, 1)
    model = tf.keras.models.load_model(model_path)
    lvl3_inorder = get_predictions(inorder_ann_sliced, n=16)
    lvl3_preorder = get_predictions(preorder_ann_sliced, n=16)
        
    encrypted_value = np.concatenate((lvl3_inorder, lvl3_preorder))

    return encrypted_value

In [60]:
inorder = []
preorder = []
encrypted_value = encrypt(input_data, salt, n = 16, user_id = 1)

## Decryption Function

In [119]:
def decrypt(encrypted_value, n, user_id): # n represents the number of nodes in the layer of the ANN
    
    mid_index = int(len(encrypted_value)/2)
    lvl3_inorder, lvl3_preorder = encrypted_value[:mid_index], encrypted_value[mid_index:]
    
    inorder_ann_input = decrypt_ann(model_path, lvl3 = lvl3_inorder, n = 16)
    preorder_ann_input = decrypt_ann(model_path, lvl3 = lvl3_preorder, n = 16)
    inorder_lvl2_decrypted = get_lvl2(inorder_ann_input)
    preorder_lvl2_decrypted = get_lvl2(preorder_ann_input)
    
    lvl1 = decrypt_binary_tree(inorder_lvl2_decrypted, preorder_lvl2_decrypted)
    
    input_array, salt = decrypt_interleave(interleaved_data = lvl1)
    
    decrypted_value = ''
    for s in input_array:
        decrypted_value += s
    
    return decrypted_value

In [120]:
decrypted_value = decrypt(encrypted_value, n = 16, user_id = 1)

In [121]:
decrypted_value

'abcdefghijklmnopqrstuvwxyz'

# Results

In [127]:
input_data = "HDNKd gKLC LFKfg  fgGAUFfdsa fCMDG"
salt = "NBPFQLLpkqIQHIz10TGG"
# input_data = "abcdefghijklmnopqrstuvwxyz"
# salt = "123456789"

In [128]:
if not os.path.exists(os.path.join(os.getcwd(), 'weights')):
    os.makedirs(os.path.join(os.getcwd(), 'weights'))        

In [129]:
inorder = []
preorder = []
encrypted_value = encrypt(input_data, salt, n = 16, user_id = 1)
print("The encrypted value is :- ")
print(encrypted_value)

The encrypted value is :- 
[[[189380.95     92746.234   160445.3      66694.84    116500.7
    45273.414    38770.625   161019.9     249999.02     98925.71
   117495.08     -4416.9297   50432.03    163458.1     175974.77
    70263.55   ]]

 [[126804.29     61865.188    84194.13     33459.055    83826.55
    41860.047    30711.07    126374.64    181938.72     73125.44
    91622.33      4963.75     39523.777    96353.016   114529.9
    35607.57   ]]

 [[220360.77    117323.79    148124.38     46257.547   131731.19
   102885.21     95251.71    196004.56    312129.12    134155.92
   225952.28     37723.84     81748.38    164900.16    217966.08
    37398.086  ]]

 [[107872.8      69946.36     63106.03     33666.766    90628.64
    88987.11    125159.02    119426.      203216.56     83052.86
   137824.36     14132.528    41068.28     77326.734   123423.49
    62915.43   ]]

 [[ 49710.035    30407.902    22858.656     8607.968    41362.46
    35526.57     33873.5      42387.19     69110.45   

In [130]:
decrypted_value = decrypt(encrypted_value, n = 16, user_id = 1)
print("The decrypted value is :- ")
print(decrypted_value)

The decrypted value is :- 
HDNKd gKLC LFKfg  fgGAUFfdsa fCMDG


In [131]:
if(input_data == decrypted_value):
    print("SUCCESS!!")

SUCCESS!!


# Testing on image

In [132]:
import cv2

In [137]:
original_img = cv2.imread("test_image.jpg", cv2.IMREAD_COLOR)

In [138]:
original_shape = original_img.shape

In [139]:
img = copy.deepcopy(original_img)

In [140]:
img = img.reshape(-1)

In [142]:
len_array = []
for val in img:
    len_array.append(len(str(val)))

In [145]:
input_data = ''
for i in img:
    input_data += str(i)

In [147]:
if not os.path.exists(os.path.join(os.getcwd(), 'weights')):
    os.makedirs(os.path.join(os.getcwd(), 'weights')) 

In [ ]:
inorder = []
preorder = []
encrypted_value = encrypt(input_data, salt, n = 16, user_id = 1)
print("The encrypted value is :- ")
print(encrypted_value)